In [52]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanIoU
import pandas as pd
import numpy as np
import openpyxl
import SimpleITK as sitk
from sklearn.model_selection import train_test_split

In [32]:
def unet_3d(input_shape):
    inputs = layers.Input(input_shape)
    
    # Encoder: Downsampling path
    # Block 1
    c1 = layers.Conv3D(32, (3, 3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv3D(32, (3, 3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling3D((2, 2, 2))(c1)
    
    # Block 2
    c2 = layers.Conv3D(64, (3, 3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv3D(64, (3, 3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling3D((2, 2, 2))(c2)

    # Block 3
    c3 = layers.Conv3D(128, (3, 3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv3D(128, (3, 3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling3D((2, 2, 2))(c3)

    # Block 4
    c4 = layers.Conv3D(256, (3, 3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv3D(256, (3, 3, 3), activation='relu', padding='same')(c4)
    p4 = layers.MaxPooling3D((2, 2, 2))(c4)

    # Bottleneck
    c5 = layers.Conv3D(512, (3, 3, 3), activation='relu', padding='same')(p4)
    c5 = layers.Conv3D(512, (3, 3, 3), activation='relu', padding='same')(c5)

    # Decoder: Upsampling path
    # Block 6
    u6 = layers.Conv3DTranspose(256, (2, 2, 2), strides=(2, 2, 2), padding='same')(c5)
    u6 = layers.concatenate([u6, c4])
    c6 = layers.Conv3D(256, (3, 3, 3), activation='relu', padding='same')(u6)
    c6 = layers.Conv3D(256, (3, 3, 3), activation='relu', padding='same')(c6)

    # Block 7
    u7 = layers.Conv3DTranspose(128, (2, 2, 2), strides=(2, 2, 2), padding='same')(c6)
    u7 = layers.concatenate([u7, c3])
    c7 = layers.Conv3D(128, (3, 3, 3), activation='relu', padding='same')(u7)
    c7 = layers.Conv3D(128, (3, 3, 3), activation='relu', padding='same')(c7)

    # Block 8
    u8 = layers.Conv3DTranspose(64, (2, 2, 2), strides=(2, 2, 2), padding='same')(c7)
    u8 = layers.concatenate([u8, c2])
    c8 = layers.Conv3D(64, (3, 3, 3), activation='relu', padding='same')(u8)
    c8 = layers.Conv3D(64, (3, 3, 3), activation='relu', padding='same')(c8)

    # Block 9
    u9 = layers.Conv3DTranspose(32, (2, 2, 2), strides=(2, 2, 2), padding='same')(c8)
    u9 = layers.concatenate([u9, c1])
    c9 = layers.Conv3D(32, (3, 3, 3), activation='relu', padding='same')(u9)
    c9 = layers.Conv3D(32, (3, 3, 3), activation='relu', padding='same')(c9)

    # Output layer: 1 output channel (for binary segmentation)
    outputs = layers.Conv3D(1, (1, 1, 1), activation='sigmoid')(c9)

    model = models.Model(inputs=[inputs], outputs=[outputs])
    
    return model

In [64]:
PROFUNDIDADE = 128 #Número de fatias/camadas na imagem 3D

ALTURA = 128 #Número de píxeis

LARGURA = 128 #Número de píxeis

CANAIS_RGB = 1 #Número de canais RGB (cores)

input_shape = (PROFUNDIDADE, ALTURA, LARGURA, CANAIS_RGB)  # Ajustar de acordo com as imagens

model = unet_3d(input_shape)

model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=[MeanIoU(num_classes=2)])

model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 128, 1)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_57 (Conv3D)  │ (None, 128, 128,  │        896 │ input_layer_3[0]… │
│                     │ 128, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_58 (Conv3D)  │ (None, 128, 128,  │     27,680 │ conv3d_57[0][0]   │
│                     │ 128, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d_12    │ (None, 64, 64,    │          0 │ conv3d_58[0][0]   │
│ (MaxPooling3D)      │ 64, 32)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_59 (Conv3D)  │ (None, 64, 64,    │     55,360 │ max_pooling3d_12… │
│                     │ 64, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_60 (Conv3D)  │ (None, 64, 64,    │    110,656 │ conv3d_59[0][0]   │
│                     │ 64, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d_13    │ (None, 32, 32,    │          0 │ conv3d_60[0][0]   │
│ (MaxPooling3D)      │ 32, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_61 (Conv3D)  │ (None, 32, 32,    │    221,312 │ max_pooling3d_13… │
│                     │ 32, 128)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_62 (Conv3D)  │ (None, 32, 32,    │    442,496 │ conv3d_61[0][0]   │
│                     │ 32, 128)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d_14    │ (None, 16, 16,    │          0 │ conv3d_62[0][0]   │
│ (MaxPooling3D)      │ 16, 128)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_63 (Conv3D)  │ (None, 16, 16,    │    884,992 │ max_pooling3d_14… │
│                     │ 16, 256)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_64 (Conv3D)  │ (None, 16, 16,    │  1,769,728 │ conv3d_63[0][0]   │
│                     │ 16, 256)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d_15    │ (None, 8, 8, 8,   │          0 │ conv3d_64[0][0]   │
│ (MaxPooling3D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_65 (Conv3D)  │ (None, 8, 8, 8,   │  3,539,456 │ max_pooling3d_15… │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_66 (Conv3D)  │ (None, 8, 8, 8,   │  7,078,400 │ conv3d_65[0][0]   │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_transpose_12 │ (None, 16, 16,    │  1,048,832 │ conv3d_66[0][0]   │
│ (Conv3DTranspose)   │ 16, 256)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_12      │ (None, 16, 16,    │          0 │ conv3d_transpose

 Total params: 22,575,329 (86.12 MB)

 Trainable params: 22,575,329 (86.12 MB)

 Non-trainable params: 0 (0.00 B)

In [65]:
df = pd.read_excel('lidc-idri-nodule-counts-6-23-2015.xlsx')

In [56]:
def load_scan(path):
    scan = sitk.ReadImage(path)
    scan_array = sitk.GetArrayFromImage(scan)
    return scan_array

In [57]:
image_paths = []
labels = []

for index, row in df.iterrows():
    scan_id = row['TCIA Patient ID']
    numb_nodules = row['Total Number of Nodules* ']
    numb_nodules_minor3mm = row['Number of Nodules >=3mm**']
    numb_nodules_bigger3mm = row['Number of Nodules <3mm***']
    
    scan_path = f'test_images/LIDC-IRDI-{scan_id}.png'
    image_paths.append(scan_path)
    labels.append((numb_nodules, numb_nodules_minor3mm, numb_nodules_bigger3mm))

In [58]:
train_paths, test_paths, train_labels, test_labels = train_test_split(image_paths, labels, test_size=0.2, random_state=42)

In [59]:
def load_data(image_paths, labels):
    images = []
    for path in image_paths:
        scan_array = load_scan(path)
        images.append(scan_array)
    return images, labels

train_images, train_labels = load_data(train_paths, train_labels)
test_images, test_labels = load_data(test_paths, test_labels)

RuntimeError: Exception thrown in SimpleITK ImageFileReader_Execute: /tmp/SimpleITK/Code/IO/src/sitkImageReaderBase.cxx:91:
sitk::ERROR: The file "test_images/LIDC-IRDI-LIDC-IDRI-0832.png" does not exist.

In [62]:
model.fit(np.array(train_images), np.array(train_labels), epochs=50, batch_size=2, validation_split=0.2)

model.evaluate(np.array(test_images), np.array(test_labels))
model.save('3d_unet_model.h5')

NameError: name 'train_images' is not defined